In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

2023-11-27 19:12:36.355005: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 19:12:46.745231: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-27 19:12:46.745459: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-11-27 19:12:46.745475: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [10]:
import os

df = pd.read_csv("Users/yc1063/fall-2023-reddit-project-team-11/data/csv/ml.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Users/yc1063/fall-2023-reddit-project-team-11/data/csv/ml.csv'

In [ ]:
df = df.sort_values(by=['subreddit', 'year', 'month'])

# Pivot the DataFrame to have 'subreddit' as columns and 'year-month' as index
df_pivot = df.pivot_table(index=['year', 'month'], columns='subreddit', values='post_count', aggfunc='sum', fill_value=0)

# Convert the index to datetime
df_pivot.index = pd.to_datetime(df_pivot.index.map(lambda x: '-'.join(map(str, x))))

# Normalize the data using Min-Max scaling
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_pivot), columns=df_pivot.columns, index=df_pivot.index)

# Prepare input sequences and labels
sequence_length = 3  # You can adjust this based on your requirements

X, y = [], []
for i in range(len(df_scaled) - sequence_length):
    X.append(df_scaled.iloc[i:i+sequence_length].values)
    y.append(df_scaled.iloc[i+sequence_length].values)

X = np.array(X)
y = np.array(y)

# Define the RNN model
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(sequence_length, len(df_scaled.columns))))
model.add(Dense(len(df_scaled.columns)))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')  # Mean Squared Error is a common loss function for regression problems

# Train the model
model.fit(X, y, epochs=50, batch_size=16, validation_split=0.1)

# Now you can use the trained model for predictions
# You might want to define a separate test set for evaluation

# For example, if you have a test set named 'X_test'
# X_test = ...

# Make predictions
predictions = model.predict(X_test)

# Inverse transform the predictions to get them back to the original scale
predictions_original_scale = scaler.inverse_transform(predictions)